---

_You are currently looking at **version 1.2** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-social-network-analysis/resources/yPcBs) course resource._

---

# Assignment 4

In [1]:
import networkx as nx
import pandas as pd
import numpy as np
import pickle

---

## Part 1 - Random Graph Identification

For the first part of this assignment you will analyze randomly generated graphs and determine which algorithm created them.

In [2]:
P1_Graphs = pickle.load(open('A4_graphs','rb'))
P1_Graphs

<br>
`P1_Graphs` is a list containing 5 networkx graphs. Each of these graphs were generated by one of three possible algorithms:
* Preferential Attachment (`'PA'`)
* Small World with low probability of rewiring (`'SW_L'`)
* Small World with high probability of rewiring (`'SW_H'`)

Anaylze each of the 5 graphs and determine which of the three algorithms generated the graph.

*The `graph_identification` function should return a list of length 5 where each element in the list is either `'PA'`, `'SW_L'`, or `'SW_H'`.*

In [6]:
def graph_identification():
    
    # Your Code Here
    method=[]
    for G in P1_Graphs:
        degrees=G.degree()
        degree_values=sorted(set(degrees.values()))
        degree_hist= [list(degrees.values()).count(i)/nx.number_of_nodes(G) for i in degree_values]
        clustering=nx.average_clustering(G)
        ASPL=nx.average_shortest_path_length(G)
        if len(degree_hist) > 10:
            method.append('PA')
        elif clustering < 0.1:
            method.append('SW_H')
        else:
            method.append('SW_L')
    return method # Your Answer Here
graph_identification()

['PA', 'SW_L', 'SW_L', 'PA', 'SW_H']

---

## Part 2 - Company Emails

For the second part of this assignment you will be workking with a company's email network where each node corresponds to a person at the company, and each edge indicates that at least one email has been sent between two people.

The network also contains the node attributes `Department` and `ManagementSalary`.

`Department` indicates the department in the company which the person belongs to, and `ManagementSalary` indicates whether that person is receiving a management position salary.

In [7]:
G = nx.read_gpickle('email_prediction.txt')
G.nodes
G.edges(data)
print(nx.info(G))

Name: 
Type: Graph
Number of nodes: 1005
Number of edges: 16706
Average degree:  33.2458


### Part 2A - Salary Prediction

Using network `G`, identify the people in the network with missing values for the node attribute `ManagementSalary` and predict whether or not these individuals are receiving a management position salary.

To accomplish this, you will need to create a matrix of node features using networkx, train a sklearn classifier on nodes that have `ManagementSalary` data, and predict a probability of the node receiving a management salary for nodes where `ManagementSalary` is missing.



Your predictions will need to be given as the probability that the corresponding employee is receiving a management position salary.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 252 with the data being the probability of receiving management salary, and the index being the node id.

    Example:
    
        1       1.0
        2       0.0
        5       0.8
        8       1.0
            ...
        996     0.7
        1000    0.5
        1001    0.0
        Length: 252, dtype: float64

In [45]:
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GridSearchCV
def salary_predictions():
    
    # Your Code Here
    df = pd.DataFrame(index=G.nodes())
    df['ManagementSalary']=pd.Series(nx.get_node_attributes(G, 'ManagementSalary'))
#     df['Department']=pd.Series(nx.get_node_attributes(G, 'Department'))
    df['Clustering']=pd.Series(nx.clustering(G))
    df['Degree']=pd.Series(nx.degree(G))
#     df['Eccentricity']=pd.Series(nx.eccentricity(G))
    df['Degree_centrality']=pd.Series(nx.degree_centrality(G))
    df['Closeness_centrality']=pd.Series(nx.closeness_centrality(G))
    df['Betweenness_centrality']=pd.Series(nx.betweenness_centrality(G,normalized=True,endpoints=False))
    df['Pagerank']=pd.Series(nx.pagerank(G,alpha=0.9))
#     df['Hubs']=pd.Series(nx.hits(G)[0])
#     df['Authorities']=pd.Series(nx.hits(G)[-1])
    df_train=df.dropna()
    df_test=df[df['ManagementSalary'].isnull()]
    features=['Clustering','Degree','Degree_centrality','Closeness_centrality','Betweenness_centrality','Pagerank']
    X_train=df_train[features]
    y_train=df_train['ManagementSalary']
    X_test=df_test[features]
    scaler = MinMaxScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    clf=MLPClassifier(random_state=0,verbose=0,solver='lbfgs').fit(X_train,y_train)
    grid_values={'hidden_layer_sizes':[[10,10],[10,5],[10,10,10]],'alpha':[5,10,100]}
    grid_clf_custom = GridSearchCV(clf, param_grid=grid_values, scoring='roc_auc')
    grid_clf_custom.fit(X_train,y_train)
#     print('Grid best score ({0}): {1}'.format('roc_auc', grid_clf_custom.best_params_))
#     print('Grid best param ({0}): {1}'.format('roc_auc', grid_clf_custom.best_score_))
    proba=grid_clf_custom.predict_proba(X_test)
    probability=pd.Series(proba[:,1],index=X_test.index)
    
    return probability# Your Answer Here
salary_predictions()

1       0.196458
2       0.538521
5       0.948663
8       0.124156
14      0.403503
18      0.220431
27      0.267239
30      0.304299
31      0.185252
34      0.102331
37      0.124154
40      0.246433
45      0.174526
54      0.190815
55      0.149887
60      0.214261
62      0.989567
65      0.502225
77      0.083951
79      0.154555
97      0.083959
101     0.095822
103     0.196476
108     0.149864
113     0.465955
122     0.089707
141     0.430010
142     0.861324
144     0.059906
145     0.327863
          ...   
913     0.073424
914     0.078534
915     0.036916
918     0.092741
923     0.043929
926     0.059912
931     0.064131
934     0.040965
939     0.039560
944     0.036916
945     0.042427
947     0.050469
950     0.159351
951     0.052223
953     0.061975
959     0.036916
962     0.038210
963     0.145299
968     0.040985
969     0.038239
974     0.045486
984     0.039560
987     0.054061
989     0.040985
991     0.040985
992     0.039550
994     0.036916
996     0.0369

### Part 2B - New Connections Prediction

For the last part of this assignment, you will predict future connections between employees of the network. The future connections information has been loaded into the variable `future_connections`. The index is a tuple indicating a pair of nodes that currently do not have a connection, and the `Future Connection` column indicates if an edge between those two nodes will exist in the future, where a value of 1.0 indicates a future connection.

In [40]:
future_connections = pd.read_csv('Future_Connections.csv', index_col=0, converters={0: eval})
future_connections.head(10)

,Future Connection
"(6, 840)",0.0
"(4, 197)",0.0
"(620, 979)",0.0
"(519, 872)",0.0
"(382, 423)",0.0
"(97, 226)",1.0
"(349, 905)",0.0
"(429, 860)",0.0
"(309, 989)",0.0
"(468, 880)",0.0


Using network `G` and `future_connections`, identify the edges in `future_connections` with missing values and predict whether or not these edges will have a future connection.

To accomplish this, you will need to create a matrix of features for the edges found in `future_connections` using networkx, train a sklearn classifier on those edges in `future_connections` that have `Future Connection` data, and predict a probability of the edge being a future connection for those edges in `future_connections` where `Future Connection` is missing.



Your predictions will need to be given as the probability of the corresponding edge being a future connection.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 122112 with the data being the probability of the edge being a future connection, and the index being the edge as represented by a tuple of nodes.

    Example:
    
        (107, 348)    0.35
        (542, 751)    0.40
        (20, 426)     0.55
        (50, 989)     0.35
                  ...
        (939, 940)    0.15
        (555, 905)    0.35
        (75, 101)     0.65
        Length: 122112, dtype: float64

In [69]:
# from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import train_test_split

def new_connections_predictions():
    
    from sklearn.ensemble import GradientBoostingClassifier
    
    future_connections['pref_attachment'] = [list(nx.preferential_attachment(G, [node_pair]))[0][2]
                                             for node_pair in future_connections.index]
    future_connections['comm_neighbors'] = [len(list(nx.common_neighbors(G, node_pair[0], node_pair[1]))) 
                                            for node_pair in future_connections.index]
    train_data = future_connections[~future_connections['Future Connection'].isnull()]
    test_data = future_connections[future_connections['Future Connection'].isnull()]
    X_train, X_test, y_train, y_test = train_test_split(train_data[['pref_attachment','comm_neighbors']].values,  train_data['Future Connection'].values)
    clf = GradientBoostingClassifier().fit(X_train,y_train)
#     y_score_lr = clf.fit(X_train, y_train).decision_function(X_test)
#     fpr_lr, tpr_lr, _ = roc_curve(y_test, y_score_lr)
#     roc_auc_lr = auc(fpr_lr, tpr_lr)
#     print(roc_auc_lr)
    preds = clf.predict_proba(test_data[['pref_attachment','comm_neighbors']].values)[:,1]
    return  pd.Series(preds, index=test_data.index)

new_connections_predictions()

(107, 348)    0.032481
(542, 751)    0.012810
(20, 426)     0.545319
(50, 989)     0.012951
(942, 986)    0.012951
(324, 857)    0.012951
(13, 710)     0.152682
(19, 271)     0.145770
(319, 878)    0.012951
(659, 707)    0.012949
(49, 843)     0.012951
(208, 893)    0.012949
(377, 469)    0.012617
(405, 999)    0.021404
(129, 740)    0.017979
(292, 618)    0.018283
(239, 689)    0.012951
(359, 373)    0.013320
(53, 523)     0.032755
(276, 984)    0.012951
(202, 997)    0.012951
(604, 619)    0.112750
(270, 911)    0.012951
(261, 481)    0.067438
(200, 450)    0.836765
(213, 634)    0.012810
(644, 735)    0.082422
(346, 553)    0.012786
(521, 738)    0.012298
(422, 953)    0.017979
                ...   
(672, 848)    0.012951
(28, 127)     0.979025
(202, 661)    0.012786
(54, 195)     0.997943
(295, 864)    0.012951
(814, 936)    0.012810
(839, 874)    0.012951
(139, 843)    0.012951
(461, 544)    0.013145
(68, 487)     0.011866
(622, 932)    0.012949
(504, 936)    0.017547
(479, 528) 